In [ ]:
%%writefile .env
ORACLE_USER=RM567189
ORACLE_PASSWORD="230795"
ORACLE_DSN=oracle.fiap.com.br:1521/orcl

Overwriting .env


In [ ]:
!pip install oracledb
from google.colab import drive
import pandas as pd
import numpy as np
import oracledb
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
drive.mount('/content/drive')

load_dotenv()

CREDENTIALS = {
'user' : os.getenv('ORACLE_USER'),
'password' : os.getenv('ORACLE_PASSWORD'),
'dsn' : os.getenv('ORACLE_DSN')
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Criando os DataFrames
load_dotenv()

CREDENTIALS = {
'user' : os.getenv('ORACLE_USER'),
'password' : os.getenv('ORACLE_PASSWORD'),
'dsn' : os.getenv('ORACLE_DSN')
}

print("📂 Lendo arquivos CSV...")
df_auxiliar_nuclea = pd.read_csv("/content/drive/MyDrive/FIAP/base_auxiliar_fiap.csv")
df_boletos_nuclea = pd.read_csv("/content/drive/MyDrive/FIAP/base_boletos_fiap.csv")

USER = CREDENTIALS['user']
PASS = CREDENTIALS["password"]
DSN = CREDENTIALS["dsn"]

📂 Lendo arquivos CSV...


In [ ]:
# Mudança das datas
print("⏳ Ligando a Máquina do Tempo...")

# Converter para datetime
colunas_data = ["dt_emissao", "dt_vencimento", "dt_pagamento"]
for col in colunas_data:
  df_boletos_nuclea[col] = pd.to_datetime(df_boletos_nuclea[col], format = '%Y-%m-%d', errors = "coerce")
df_boletos_nuclea.dtypes

# Configurar semente para resultados reproduzíveis
np.random.seed(42)

# Isso vai espalhar os boletos de Maio/2024 até o início de 2023
dias_para_voltar = np.random.randint(0, 500, size=len(df_boletos_nuclea))

# Ajustar Vencimento
df_boletos_nuclea['dt_vencimento'] = df_boletos_nuclea['dt_vencimento'] - pd.to_timedelta(dias_para_voltar, unit='d')

# Ajustar Emissão (Para ser coerente, a emissão acontece 10 a 60 dias antes do vencimento)
dias_prazo = np.random.randint(10, 60, size=len(df_boletos_nuclea))
df_boletos_nuclea['dt_emissao'] = df_boletos_nuclea['dt_vencimento'] - pd.to_timedelta(dias_prazo, unit='d')

# Ajustar as colunas que não possuem valores
df_boletos_nuclea.loc[
    (df_boletos_nuclea["vlr_baixa"].isnull()) & ~(df_boletos_nuclea["tipo_baixa"].isnull()),
    "tipo_baixa"
].unique()
df_boletos_nuclea["vlr_baixa"] = df_boletos_nuclea["vlr_baixa"].fillna(0.0)
df_boletos_nuclea["tipo_baixa"] = df_boletos_nuclea["tipo_baixa"].fillna("PENDENTE")


⏳ Ligando a Máquina do Tempo...


In [ ]:
# 3. Ajustar Pagamento (Para manter a consistência dos dias de atraso originais)
# Se o cliente pagou com 5 dias de atraso, a data de pagamento deve ser Vencimento Novo + 5 dias
# Se inadimplente (data nula), continua nula.
df_boletos_nuclea["nr_dias_atraso"] = (df_boletos_nuclea["dt_pagamento"] - df_boletos_nuclea["dt_vencimento"]).dt.days
df_boletos_nuclea["nr_dias_atraso"] = df_boletos_nuclea["nr_dias_atraso"].apply(lambda x: 0 if x < 0 else x)
df_boletos_nuclea.loc[df_boletos_nuclea["dt_pagamento"].isnull(), "nr_dias_atraso"] = (datetime.now() - df_boletos_nuclea["dt_vencimento"]).dt.days

df_boletos_nuclea["alvo_inadimplencia"] = np.where(df_boletos_nuclea["nr_dias_atraso"] > 0, 1, 0)

print(f"   -> Novo Período: De {df_boletos_nuclea['dt_vencimento'].min().date()} até {df_boletos_nuclea['dt_vencimento'].max().date()}")

# Seleção de Colunas na ordem da tabela T_BF_BOLETO
dados_insert = df_boletos_nuclea[[
    'id_boleto',
    'id_pagador',
    'id_beneficiario',
    'vlr_nominal',
    'vlr_baixa',
    'dt_emissao',
    'dt_vencimento',
    'dt_pagamento',
    'tipo_baixa',
    'nr_dias_atraso',
    'alvo_inadimplencia'
]].values.tolist()

   -> Novo Período: De 2022-12-19 até 2024-05-30


In [ ]:
df_auxiliar_nuclea["cd_cnae_prin"] = df_auxiliar_nuclea["cd_cnae_prin"].fillna(0000000)
df_auxiliar_nuclea["cd_cnae_prin"] = df_auxiliar_nuclea["cd_cnae_prin"].astype(int).astype(str)

df_auxiliar_nuclea["uf"] = df_auxiliar_nuclea["uf"].fillna("ND")
df_auxiliar_nuclea["uf"] = df_auxiliar_nuclea["uf"].str.strip().str.upper()

df_auxiliar_nuclea["score_quantidade_v2"] = df_auxiliar_nuclea["score_quantidade_v2"].fillna(0.0)
df_auxiliar_nuclea["score_materialidade_v2"] = df_auxiliar_nuclea["score_materialidade_v2"].fillna(0.0)

In [ ]:
# Inserindo os dados no Banco Oracle
print("\n🔌 Conectando ao Banco de Dados...")

try:
    with oracledb.connect(user=USER, password=PASS, dsn=DSN) as connection:
      cursor = connection.cursor()
      print("🔌Conectou")

      # Limpando as tabelas
      cursor.execute("TRUNCATE TABLE T_BF_BOLETO")
      cursor.execute("TRUNCATE TABLE T_BF_EMPRESA")
      print("🧹 Tabela limpa.")

      # EMPRESAS
      colunas_empresas_sql = ['id_cnpj', 'cd_cnae_prin', 'uf', 'score_materialidade_v2','score_quantidade_v2']
      dados_empresas = df_auxiliar_nuclea[colunas_empresas_sql].values.tolist()

      sql_insert_empresas = """
          INSERT INTO T_BF_EMPRESA (id_empresa, cd_cnae, sg_uf, vl_score_materialidade, vl_score_quantidade)
          VALUES (:1, :2, :3, :4, :5)
      """
      cursor.executemany(sql_insert_empresas, dados_empresas)
      print(f"   -> {cursor.rowcount} empresas inseridas.")

      # TABELA BOLETOS
      colunas_boletos_sql = [
          'id_boleto', 'id_pagador', 'id_beneficiario', 'vlr_nominal', 'vlr_baixa',
          'dt_emissao', 'dt_vencimento', 'dt_pagamento', 'tipo_baixa', 'nr_dias_atraso', 'alvo_inadimplencia'
      ]
      dados_boletos = df_boletos_nuclea[colunas_boletos_sql].values.tolist()

      sql_insert_boletos = """
          INSERT INTO T_BF_BOLETO
          (id_boleto, id_pagador, id_beneficiario, vl_nominal, vl_baixa,
          dt_emissao, dt_vencimento, dt_pagamento, tp_baixa, nr_dias_atraso, vl_inadimplencia)
          VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11)
      """
      cursor.executemany(sql_insert_boletos, dados_boletos)
      print(f"   -> {cursor.rowcount} boletos inseridos.")

      # Commit final
      connection.commit()
      print("\n✅ CARGA FINALIZADA COM SUCESSO!")

except oracledb.Error as e:
    print(f"❌ Erro fatal na conexão ou execução: {e}")


🔌 Conectando ao Banco de Dados...
🔌Conectou
🧹 Tabela limpa.
   -> 4612 empresas inseridas.
   -> 7118 boletos inseridos.

✅ CARGA FINALIZADA COM SUCESSO!
